<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Edited by Sergey Kolchenko (@KolchenkoSergey). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

## <center>Assignment #6
### <center> Beating baselines in "How good is your Medium article?"
    
<img src='../../img/medium_claps.jpg' width=40% />


[Competition](https://www.kaggle.com/c/how-good-is-your-medium-article). The task is to beat "A6 baseline" (~1.45 Public LB score). Do not forget about our shared ["primitive" baseline](https://www.kaggle.com/kashnitsky/ridge-countvectorizer-baseline) - you'll find something valuable there.

**Your task:**
 1. "Freeride". Come up with good features to beat the baseline "A6 baseline" (for now, public LB is only considered)
 2. You need to name your [team](https://www.kaggle.com/c/how-good-is-your-medium-article/team) (out of 1 person) in full accordance with the [course rating](https://drive.google.com/open?id=19AGEhUQUol6_kNLKSzBsjcGUU3qWy3BNUg8x8IFkO3Q). You can think of it as a part of the assignment. 16 credits for beating the mentioned baseline and correct team naming.
 
*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a6__*

In [44]:
import os
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack, vstack, save_npz, load_npz
from sklearn.linear_model import Ridge, RidgeCV, SGDRegressor
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, TimeSeriesSplit
import gc
import warnings
import gensim
from matplotlib import pyplot as plt
warnings.filterwarnings('ignore')
from IPython.core.debugger import set_trace
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import xgboost as xgb
import re
from langdetect import detect
%matplotlib inline

The following code will help to throw away all HTML tags from an article content.

Supplementary function to read a JSON line without crashing on escape characters.

In [7]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [8]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

In [9]:
def extract_features(path_to_data):
    
    content_list = [] 
    published_list = [] 
    title_list = []
    author_list = []
    domain_list = []
    tags_list = []
    url_list = []
    images_list = []
    frames_list = []
    lang_list = []
    
    with open(path_to_data, encoding='utf-8') as inp_json_file:
        for line in inp_json_file:
            json_data = read_json_line(line)
            content = json_data['content'].replace('\n', ' ').replace('\r', ' ')
            content_no_html_tags = strip_tags(content)
            content_list.append(content_no_html_tags)
            published = json_data['published']['$date']
            published_list.append(published) 
            title = json_data['meta_tags']['title'].split('\u2013')[0].strip() #'Medium Terms of Service – Medium Policy – Medium'
            title_list.append(title) 
            author = json_data['meta_tags']['author'].strip()
            author_list.append(author) 
            domain = json_data['domain']
            domain_list.append(domain)
            url = json_data['url']
            url_list.append(url)
            
            tags_str = []
            soup = BeautifulSoup(content, 'lxml')
            try:
                tag_block = soup.find('ul', class_='tags')
                tags = tag_block.find_all('a')
                for tag in tags:
                    tags_str.append(tag.text.translate({ord(' '):None, ord('-'):None}))
                tags = ' '.join(tags_str)
            except Exception:
                tags = 'None'
            tags_list.append(tags)
            
            frames = re.findall(r'iframeContainer', content)
            images = re.findall(r'<img class=', content)

            images_list.append(len(images))
            frames_list.append(len(frames))
            try:
                lang = detect(content_no_html_tags)
            except Exception:      
                lang = 'undefined'   
            lang_list.append(lang)
            
    return content_list, published_list, title_list, author_list, domain_list, tags_list, url_list, images_list, frames_list, lang_list

In [10]:
PATH_TO_DATA = '../../data/kaggle_medium' # modify this if you need to

In [75]:
%%time
content_list, published_list, title_list, author_list, domain_list, tags_list, url_list, images_list, frames_list, lang_list = extract_features(os.path.join(PATH_TO_DATA, 'train.json'))
train = pd.DataFrame()
train['content'] = content_list
train['published'] = pd.to_datetime(published_list, format='%Y-%m-%dT%H:%M:%S.%fZ')
train['title'] = title_list
train['author'] = author_list
train['domain'] = domain_list
train['tags'] = tags_list
train['length'] = train['content'].apply(len)
train['url'] = url_list
train['images'] = images_list
train['frames'] = frames_list
train['lang'] = lang_list

content_list, published_list, title_list, author_list, domain_list, tags_list, url_list, images_list, frames_list, lang_list = extract_features(os.path.join(PATH_TO_DATA, 'test.json'))
test = pd.DataFrame()
test['content'] = content_list
test['published'] = pd.to_datetime(published_list, format='%Y-%m-%dT%H:%M:%S.%fZ')
test['title'] = title_list
test['author'] = author_list
test['domain'] = domain_list
test['tags'] = tags_list
test['length'] = test['content'].apply(len)
test['url'] = url_list
test['images'] = images_list
test['frames'] = frames_list
test['lang'] = lang_list
del content_list, published_list, title_list, author_list, domain_list, tags_list, url_list, images_list, frames_list, lang_list

Wall time: 1h 1min 48s


In [76]:
#train.to_pickle('train.pkl')
#test.to_pickle('test.pkl')

In [11]:
train = pd.read_pickle('train.pkl')
test = pd.read_pickle('test.pkl')

In [12]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_log1p_recommends.csv'), index_col='id')
y_train = train_target['log_recommends'].values

In [77]:
df = pd.concat([train, test])
df.reset_index(inplace=True)

### Feature Engineering
Building datetime features

In [13]:
idx_split = y_train.shape[0]
idx_split

62313

In [14]:
%%time
cv = CountVectorizer(max_features=50000, min_df = 0.1, max_df = 0.8)
sparse_train = cv.fit_transform(train['content'])
sparse_test  = cv.transform(test['content'])
full_sparse_data =  vstack([sparse_train, sparse_test])
corpus_data_gensim = gensim.matutils.Sparse2Corpus(full_sparse_data, documents_columns=False)
del sparse_train, sparse_test, full_sparse_data

lda = gensim.models.LdaModel(corpus_data_gensim, num_topics = 30)

def document_to_lda_features(lda_model, document):
    topic_importances = lda.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]

lda_features = list(map(lambda doc:document_to_lda_features(lda, doc),corpus_data_gensim))
data_pd_lda_features = pd.DataFrame(lda_features)

df_lda_train = data_pd_lda_features.iloc[:idx_split, :]
df_lda_test = data_pd_lda_features.iloc[idx_split:, :]
del data_pd_lda_features

Wall time: 5min 56s


In [15]:
train = pd.concat([train, df_lda_train], axis=1)
test = pd.concat([test, df_lda_test.reset_index(drop=True)], axis=1)
del df_lda_train, df_lda_test

In [16]:
train['target'] = y_train
train.sort_values('published', inplace=True)
train.reset_index(drop=True, inplace=True)
y_train = train['target'].values
train.drop('target', axis=1, inplace=True)
train.head()

,content,published,title,author,domain,tags,length,url,images,frames,...,20,21,22,23,24,25,26,27,28,29
0,Susan BrattonTrusted Hot Sex Advisor To Millio...,1970-01-01 00:00:00.001,Saving Your Marriage By Watching Steamy Sex Ed...,Susan Bratton,medium.com,Lovemaking Sex SexPositions EarlyBird SexEdVideos,5473,http://personallifemedia.com/2017/01/saving-ma...,6,0,...,0.000090,0.000090,0.028759,0.000090,0.147627,0.000090,0.000090,0.000090,0.000090,0.000090
1,"Ryo OoishiDec 31, 1969やってよかった中学受験明日から息子の中学受験がは...",1970-01-01 00:00:00.001,やってよかった中学受験,Ryo Ooishi,medium.com,,5325,https://medium.com/@ooishi/%E3%82%84%E3%81%A3%...,0,0,...,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111
2,なぞちゅう仮面ライダーとかスーパー戦隊を愛する30代。特撮はたしなむ程度（自称）色々なもの、...,1970-01-18 03:21:32.400,はてなブログに書いた今年の手帳のお話,なぞちゅう,medium.com,徒然日記 手帳 ブログ,2487,http://nazoblackrx.hatenablog.com/entry/2016/1...,1,0,...,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
3,"Internet Corporation LLCDec 8, 1987SPECIAL NOT...",1987-12-08 21:45:00.000,Internet Corporation LLC to Acquire Early Clue...,Internet Corporation LLC,medium.com,SocialMedia EarlyClues InternetCorporationLlc,11285,https://medium.com/the-internet-corporation/de...,13,0,...,0.431693,0.000072,0.019082,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,0.304391
4,"Mackenzie OldridgeDec 29, 2003g sowtwaretradin...",2003-12-29 17:00:00.000,g sowtwaretrading botMoneyMoneyMakeGetting To ...,Mackenzie Oldridge,medium.com,Finance Trading,12541,http://www.investopedia.com/articles/optioninv...,2,0,...,0.452453,0.000048,0.000048,0.000048,0.000048,0.000048,0.310134,0.000048,0.000048,0.000048


In [17]:
%%time
vectorizer_content = TfidfVectorizer(ngram_range=(1, 2), max_features = 100000, stop_words='english')
vectorizer_title = TfidfVectorizer(ngram_range=(1, 2), max_features = 100000, stop_words='english')
vectorizer_tags = TfidfVectorizer(max_features = 10000, stop_words='english')
content_train = vectorizer_content.fit_transform(train['content'])
title_train = vectorizer_title.fit_transform(train['title'])
tags_train = vectorizer_tags.fit_transform(train['tags'])
content_test = vectorizer_content.transform(test['content'])
title_test = vectorizer_title.transform(test['title'])
tags_test = vectorizer_tags.transform(test['tags'])

Wall time: 6min 8s


In [26]:
del vectorizer_content, vectorizer_title, vectorizer_tags

In [20]:
df = pd.concat([train, test])
df.reset_index(inplace=True, drop=True)

In [22]:
df['year'] = df['published'].apply(lambda x: x.year)
df['weekend'] = df['published'].apply(lambda x: x.dayofweek >= 5)
df['weekend'].replace({True : 1, False : 0}, inplace=True)
df['dow_sin'] = df['published'].apply(lambda x: np.sin(2*np.pi*x.dayofweek/7))
df['dow_cos'] = df['published'].apply(lambda x: np.cos(2*np.pi*x.dayofweek/7))
df['month_sin'] = df['published'].apply(lambda x: np.sin(2*np.pi*x.month/12))
df['month_cos'] = df['published'].apply(lambda x: np.cos(2*np.pi*x.month/12))
df['hour_sin'] = df['published'].apply(lambda x: np.sin(2*np.pi*x.hour/24))
df['hour_cos'] = df['published'].apply(lambda x: np.cos(2*np.pi*x.hour/24))
df['hour'] = df['published'].apply(lambda x: x.hour)
df['morning'] = ((df['hour'] >= 7) & (df['hour'] <= 11)).astype('int')
df['afternoon'] = ((df['hour'] >= 12) & (df['hour'] <= 18)).astype('int')
df['evening'] = ((df['hour'] >= 19) & (df['hour'] <= 23)).astype('int')
df['night'] = ((df['hour'] >= 0) & (df['hour'] <= 6)).astype('int')

Misc. features

In [23]:
counts = df['author'].value_counts()
df['author'] = df['author'].replace(counts[counts <= 5].index.values, 'other')
df['number_of_tags'] = df['tags'].apply(lambda x: len(x.split()))
counts = df['domain'].value_counts()
df['domain'] = df['domain'].replace(counts[counts <= 100].index.values, 'other')
del counts
df.head()

,content,published,title,author,domain,tags,length,url,images,frames,...,month_sin,month_cos,hour_sin,hour_cos,hour,morning,afternoon,evening,night,number_of_tags
0,Susan BrattonTrusted Hot Sex Advisor To Millio...,1970-01-01 00:00:00.001,Saving Your Marriage By Watching Steamy Sex Ed...,Susan Bratton,medium.com,Lovemaking Sex SexPositions EarlyBird SexEdVideos,5473,http://personallifemedia.com/2017/01/saving-ma...,6,0,...,5.000000e-01,0.866025,0.000000,1.000000,0,0,0,0,1,5
1,"Ryo OoishiDec 31, 1969やってよかった中学受験明日から息子の中学受験がは...",1970-01-01 00:00:00.001,やってよかった中学受験,other,medium.com,,5325,https://medium.com/@ooishi/%E3%82%84%E3%81%A3%...,0,0,...,5.000000e-01,0.866025,0.000000,1.000000,0,0,0,0,1,0
2,なぞちゅう仮面ライダーとかスーパー戦隊を愛する30代。特撮はたしなむ程度（自称）色々なもの、...,1970-01-18 03:21:32.400,はてなブログに書いた今年の手帳のお話,other,medium.com,徒然日記 手帳 ブログ,2487,http://nazoblackrx.hatenablog.com/entry/2016/1...,1,0,...,5.000000e-01,0.866025,0.707107,0.707107,3,0,0,0,1,3
3,"Internet Corporation LLCDec 8, 1987SPECIAL NOT...",1987-12-08 21:45:00.000,Internet Corporation LLC to Acquire Early Clue...,other,medium.com,SocialMedia EarlyClues InternetCorporationLlc,11285,https://medium.com/the-internet-corporation/de...,13,0,...,-2.449294e-16,1.000000,-0.707107,0.707107,21,0,0,1,0,3
4,"Mackenzie OldridgeDec 29, 2003g sowtwaretradin...",2003-12-29 17:00:00.000,g sowtwaretrading botMoneyMoneyMakeGetting To ...,other,medium.com,Finance Trading,12541,http://www.investopedia.com/articles/optioninv...,2,0,...,-2.449294e-16,1.000000,-0.965926,-0.258819,17,0,1,0,0,2


In [25]:
df.drop(['content', 'title', 'tags', 'length', 'published', 'hour', 'url'], axis=1, inplace=True)
df_dummies = pd.get_dummies(df, columns = ['author', 'domain', 'images', 'frames', 'lang', 'year', 'number_of_tags'])

In [33]:
X_train = csr_matrix(hstack([df_dummies.iloc[:idx_split,:],
                             content_train,
                             title_train,
                             tags_train]))
X_test = csr_matrix(hstack([df_dummies.iloc[idx_split:,:],
                             content_test,
                             title_test,
                             tags_test]))

In [34]:
save_npz('train.npz', X_train)
save_npz('test.npz', X_test)

In [4]:
X_train = load_npz('train.npz')
X_test = load_npz('test.npz')

In [35]:
X_ttrain, X_valid, y_ttrain, y_valid = train_test_split(X_train, y_train, test_size=.25)

### Ridge model

In [36]:
ridge = Ridge()

In [37]:
%%time
ridge.fit(X_ttrain, y_ttrain)

Wall time: 26.7 s


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [38]:
mean_absolute_error(y_valid, ridge.predict(X_valid))

1.0742546550289112

In [49]:
cv = TimeSeriesSplit(n_splits=5)
param_grid = {'alpha':np.logspace(-4, 4, 10)}
ridgeSearch = RandomizedSearchCV(ridge, param_grid, scoring='neg_mean_absolute_error', verbose=1, cv=cv)

In [53]:
%%time
ridgeSearch.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 27.2min finished


Wall time: 27min 42s


RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
          error_score='raise-deprecating',
          estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'alpha': array([  1.00000e-04,   7.74264e-04,   5.99484e-03,   4.64159e-02,
         3.59381e-01,   2.78256e+00,   2.15443e+01,   1.66810e+02,
         1.29155e+03,   1.00000e+04])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=1)

In [54]:
ridgeSearch.best_params_

{'alpha': 2.7825594022071258}

### LightGBM model

In [51]:
train_lgb = lgb.Dataset(X_train, label=y_train)
params = {'bagging_fraction' : [.7, .85, 1],
          'bagging_freq' : [0, 25, 50],
          'num_leaves' : [31],
          'n_estimators' : [400],
          'learning_rate' : [.01],
          'metric' : ['mean_absolute_error'],
          'objective' : ['mean_absolute_error'],
          'num_iterations' : [700]}
lgb_mdl = lgb.LGBMRegressor()
lgbSearch = RandomizedSearchCV(lgb_mdl, params, cv=cv, verbose=1)

In [52]:
%%time
lgbSearch.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 474.2min finished


Wall time: 8h 12min 20s


RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
          error_score='raise-deprecating',
          estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'bagging_fraction': [0.7, 0.85, 1], 'bagging_freq': [0, 25, 50], 'num_leaves': [31], 'n_estimators': [400], 'learning_rate': [0.01], 'metric': ['mean_absolute_error'], 'objective': ['mean_absolute_error'], 'num_iterations': [700]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, ver

In [56]:
lgbSearch.best_params_

{'bagging_fraction': 0.7,
 'bagging_freq': 25,
 'learning_rate': 0.01,
 'metric': 'mean_absolute_error',
 'n_estimators': 400,
 'num_iterations': 700,
 'num_leaves': 31,
 'objective': 'mean_absolute_error'}

**Train the same Ridge with all available data, make predictions for the test set and form a submission file.**

In [57]:
%%time
mdl = Ridge(2.7825594022071258)
mdl.fit(X_train, y_train)
mdl_pred = mdl.predict(X_test)

Wall time: 38.7 s


In [58]:
mdl_lgb = lgb.LGBMRegressor(**lgbSearch.best_params_)
mdl_lgb.fit(X_train, y_train)
mdl_lgb_pred = mdl_lgb.predict(X_test)

In [59]:
ensemble_pred = .5*mdl_pred + .5*mdl_lgb_pred

In [60]:
final_pred = ensemble_pred + 4.33328 - np.mean(ensemble_pred)

In [40]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join(PATH_TO_DATA, 
                                                      'sample_submission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='id')
    
    submission['log_recommends'] = prediction
    submission.to_csv(filename)

In [61]:
write_submission_file(final_pred, os.path.join(PATH_TO_DATA,
                                                    'assignment6_medium_submission.csv'))

**Now's the time for dirty Kaggle hacks. Form a submission file with all zeros. Make a submission. What do you get if you think about it? How is it going to help you with modifying your predictions?**

In [52]:
write_submission_file(np.zeros_like(ridge_test_pred), 
                      os.path.join(PATH_TO_DATA,
                                   'medium_all_zeros_submission.csv'))

**Modify predictions in an appropriate way (based on your all-zero submission) and make a new submission.**

In [51]:
ridge_test_pred_modif = ridge_test_pred # You code here

In [ ]:
write_submission_file(ridge_test_pred_modif, 
                      os.path.join(PATH_TO_DATA,
                                   'assignment6_medium_submission_with_hack.csv'))

That's it for the assignment. Much more credits will be given to the winners in this competition, check [course roadmap](https://mlcourse.ai/roadmap). Do not spoil the assignment and the competition - don't share high-performing kernels (with MAE < 1.5).

Some ideas for improvement:

- Engineer good features, this is the key to success. Some simple features will be based on publication time, authors, content length and so on
- You may not ignore HTML and extract some features from there
- You'd better experiment with your validation scheme. You should see a correlation between your local improvements and LB score
- Try TF-IDF, ngrams, Word2Vec and GloVe embeddings
- Try various NLP techniques like stemming and lemmatization
- Tune hyperparameters. In our example, we've left only 50k features and used C=1 as a regularization parameter, this can be changed
- SGD and Vowpal Wabbit will learn much faster
- Play around with blending and/or stacking. An intro is given in [this Kernel](https://www.kaggle.com/kashnitsky/ridge-and-lightgbm-simple-blending) by @yorko 
- In our course, we don't cover neural nets. But it's not obliged to use GRUs/LSTMs/whatever in this competition.

Good luck!

<img src='../../img/kaggle_shakeup.png' width=50%>